In [1]:
# -*- coding: utf-8 -*-

import os
os.environ['CUDA_VISIBLE_DEVICES'] = "2"

import tensorflow as tf
import numpy as np
import time

def initialize_session():
    config = tf.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 0.4
    return tf.Session(config=config)

/etc/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
##################################################
BATCH_SIZE = 100
class_size = 4
learning_rate = 0.0005   # Learning rate
use_clip = True         # Gradient clipping 쓸지 여부
train_keep_prob = 0.7
max_len = 100           # sequence 단어 수 제한
max_vocab = 50000       # maximum 단어 개수
emb_dim = 256            # 단어 embedding dimension
hidden_dim = 256        # RNN hidden dim

train_acc_summary = [] # 학습용 데이터의 정확도를 중간 중간 기록한다
test_acc_summary = []  # 테스트용 데이터의 정확도를 중간 중간 기록한다
train_loss_summary = [] # 학습용 데이터의 정확도를 중간 중간 기록한다
test_loss_summary = []  # 테스트용 데이터의 정확도를 중간 중간 기록한다
hypothesis_summary = []
##################################################

from genre_data_loader_merge import movie_data
data = movie_data(data_type='both', max_vocab=max_vocab, max_len=max_len)
#from plot_loader import plot_data
#data = plot_data(max_vocab=max_vocab, max_len=max_len)


train_total_batch = int(data.train_size / BATCH_SIZE)
test_total_batch = int(data.test_size / BATCH_SIZE)

from genre_model_resnet_style import CNN_Model
cnn_model = CNN_Model(use_clip=use_clip, class_size=class_size, learning_rate=learning_rate)

from genre_movie_rnn_test import LSTM_Model
END_TOKEN = "<eos>"
lstm_model = LSTM_Model(max_len=max_len,
              emb_dim=emb_dim,
              hidden_dim=hidden_dim,
              vocab_size=max_vocab,
              class_size=class_size,
              use_clip=True, learning_rate=learning_rate, end_token=data.w2idx[END_TOKEN])


sess = initialize_session()
sess.run(tf.global_variables_initializer())

print(test_total_batch)

def test_model():
    test_loss, test_acc = 0, 0

    for _ in range(test_total_batch):
        test_x_img, test_x_txt, length, test_y = data.get_test(BATCH_SIZE)
        loss, acc = sess.run([lstm_model.loss, lstm_model.accuracy], feed_dict={lstm_model.x: test_x_txt, lstm_model.x_len: length, lstm_model.y: test_y})
        
        #loss, acc = sess.run([model.loss, model.accuracy],
        #                      feed_dict={model.x_image: test_x, model.y_label: test_y, model.keep_prob : 1.0, model.is_training:False})
        #test_ids, length, label = data.get_test(BATCH_SIZE)
        #loss, acc = sess.run([model.loss, model.accuracy], feed_dict={model.x: test_ids, model.x_len: length, model.y: label})

        test_loss += loss
        test_acc += acc
        
    return test_loss/test_total_batch, test_acc/test_total_batch
    #print(" * test loss: {:.3f} | test acc: {:.3f}\n".format(test_loss / test_cnt, sess.run(model.accuracy, feed_dict={model.x_image: test_x, model.y_label: test_label})))

Using TensorFlow backend.


conv1 Tensor("MaxPool:0", shape=(?, 67, 45, 64), dtype=float32)
conv2 Tensor("dropout/mul:0", shape=(?, 33, 22, 128), dtype=float32)
conv3 Tensor("dropout_1/mul:0", shape=(?, 16, 11, 256), dtype=float32)
x_image Tensor("Mean:0", shape=(?, 256), dtype=float32)
Tensor("Placeholder_3:0", shape=(?, 100), dtype=int32)
Tensor("Placeholder_4:0", shape=(?,), dtype=int32)
Tensor("Placeholder_5:0", shape=(?,), dtype=int32)
12


In [3]:
for epoch in range(20):
    
    train_avg_loss, train_avg_acc = 0, 0
    
    for it in range(train_total_batch):
        
        #train_x_img, train_x_txt, train_x_txt_length, train_y
        train_x_img, train_x_txt, length, train_y = data.get_train(BATCH_SIZE)
        
        #loss, acc, _, hypothesis_ = sess.run([model.loss, model.accuracy, model.update, model.genre_prob], 
        #                                     feed_dict={model.x_image: train_x, model.y_label: train_y, model.keep_prob:train_keep_prob, model.is_training:True})
        
        loss, acc, _, = sess.run([lstm_model.loss, lstm_model.accuracy, lstm_model.update],
                            feed_dict={lstm_model.x: train_x_txt, lstm_model.x_len: length, lstm_model.y: train_y, lstm_model.keep_prob: 0.75})
        
        #train_ids, length, label = data.get_train(BATCH_SIZE)
        
        #loss, acc, _, = sess.run([model.loss, model.accuracy, model.update],
        #                    feed_dict={model.x: train_ids, model.x_len: length, model.y: label, model.keep_prob: 0.75})
        
        train_avg_loss += loss / train_total_batch
        train_avg_acc += acc / train_total_batch
        
        if it % 10 == 0 and it > 0:
            #model.save(sess)
            print("*Model Saved train_loss: {:.4f}, train_acc: {:.4f}".format(loss/BATCH_SIZE, acc))
        
    test_avg_loss, test_avg_acc = test_model()
    
    train_acc_summary.append(train_avg_acc)
    test_acc_summary.append(test_avg_acc)
    train_loss_summary.append(train_avg_loss)
    test_loss_summary.append(test_avg_loss)
    #hypothesis_summary.append(hypothesis_)
    
    print("epoch {} - train_loss: {:.4f}, train_acc: {:.4f}, test_loss: {:.4f}, test_acc: {:.4f}"
          .format(epoch+1, train_avg_loss, train_avg_acc, test_avg_loss, test_avg_acc))  
    
    #print("epoch {} - train_loss: {:.4f}, train_acc: {:.4f}"
    #      .format(epoch+1, train_avg_loss, train_avg_acc))  

InternalError: Blas GEMM launch failed : a.shape=(100, 512), b.shape=(512, 1024), m=100, n=1024, k=512
	 [[Node: rnn/while/basic_lstm_cell/MatMul = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](rnn/while/basic_lstm_cell/concat, rnn/while/basic_lstm_cell/MatMul/Enter)]]
	 [[Node: Mean_4/_33 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_1006_Mean_4", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'rnn/while/basic_lstm_cell/MatMul', defined at:
  File "/etc/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/etc/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/etc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 1

In [ ]:
print('Max Train Accuracy {:.2f}%'.format(np.max(train_acc_summary)))
print('Max Test Accuracy {:.2f}%'.format(np.max(test_acc_summary)))

import matplotlib.pyplot as plt

# Get the figure and the axes
fig, (acc, loss) = plt.subplots(nrows=1,ncols=2, sharey=False, figsize=(15, 5))

# 첫번째 그래프
acc.plot(train_acc_summary, 'r', label='train')
acc.plot(test_acc_summary, 'b', label='test')
acc.legend(loc='upper left', frameon=False)
acc.set(title='Accuracy', xlabel='Epoch', ylabel='Accuracy')

# 두번째 그래프
loss.plot(train_loss_summary, 'r', label='train')
loss.plot(test_loss_summary, 'b', label='test')
loss.legend(loc='upper left', frameon=False)
loss.set(title='Loss', xlabel='Epoch', ylabel='Accuracy')

# Title the figure
fig.suptitle('TEST', fontsize=14, fontweight='bold')

In [ ]:
#model.restore(sess)

In [ ]:
#test_avg_loss, test_avg_acc = test_model()
#test_avg_acc

In [ ]:
hypothesis_summary[-1]

In [ ]:
w = hypothesis_summary[-1]
print(w.shape)
w = w.reshape(134,91,3,4)

w_min, w_max = np.min(w), np.max(w)

classes = ['Romance', 'Horror', 'Adventure', 'Documentary']
for i in range(4):
    plt.subplot(2,5,i+1)
    
    wimg=255.0*(w[:,:,:,i].squeeze() - w_min)/(w_max-w_min)
    plt.imshow(wimg.astype('uint8'))
    plt.axis('off')
    plt.title(classes[i])

In [ ]:
w = best_softmax.Weights[:-1, :]
w = w.reshape(32,32,3,10)

w_min, w_max = np.min(w), np.max(w)

classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for i in range(10):
    plt.subplot(2,5,i+1)
    
    wimg=255.0*(w[:,:,:,i].squeeze() - w_min)/(w_max-w_min)
    plt.imshow(wimg.astype('uint8'))
    plt.axis('off')
    plt.title(classes[i])

In [ ]:
tt = tf.nn.softmax()